# Cloud Firestore

> [Cloud Firestore](https://cloud.google.com/firestore) is a serverless document-oriented database that scales to meet any demand. Extend your database application to build AI-powered experiences leveraging Firestore's Langchain integrations.

This notebook goes over how to use [Cloud Firestore](https://cloud.google.com/firestore) to to store vectors and query them using the `FirestoreVectorStore` class.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-firestore-python/blob/main/docs/vectorstores.ipynb)

## Before You Begin

To run this notebook, you will need to do the following:
* [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
* [Create a Firestore database](https://cloud.google.com/firestore/docs/manage-databases)

After confirmed access to database in the runtime environment of this notebook, filling the following values and run the cell before running example scripts.

In [ ]:
# @markdown Please specify a source for demo purpose.
SOURCE = "test"  # @param {type:"Query"|"CollectionGroup"|"DocumentReference"|"string"}

### 🦜🔗 Library Installation

The integration lives in its own `langchain-google-firestore` package, so we need to install it. For this notebook, we will also install `langchain-google-genai` to use Google Generative AI embeddings.

In [ ]:
%pip install -upgrade --quiet langchain-google-firestore langchain-google-genai

**Colab only**: Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [1]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [10]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Updated property [core/project].


### 🔐 Authentication

Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

- If you are using Colab to run this notebook, use the cell below and continue.
- If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth

auth.authenticate_user()

### API Enablement
The `langchain-google-firestore` package requires that you [enable the Firestore Admin API](https://console.cloud.google.com/flows/enableapi?apiid=firestore.googleapis.com) in your Google Cloud Project.

In [ ]:
# enable Firestore Admin API
!gcloud services enable firestore.googleapis.com

# Basic Usage

## Add Embeddings

`FirestoreVectroStore` allows you to store new vectors in a Firestore database. You can use it to store embeddings from any model, including those from Google Generative AI.

In [11]:
from langchain_google_firestore.vectorstores import FirestoreVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

from google.api_core.client_options import ClientOptions
from google.cloud import firestore
 

embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key="",
)

client_options = ClientOptions(api_endpoint="")
client = firestore.Client(client_options=client_options)

# Sample data
fruits_texts = ['{"name": "apple"}', '{"name": "banana"}', '{"name": "orange"}']
fruits_docs = [Document(page_content=fruit) for fruit in fruits_texts]

# Create a vector store
vector_store = FirestoreVectorStore(
    client=client,
    source="fruits",
    embedding=embedding,
)

# Add the fruits to the vector store
vector_store.add_texts(fruits_texts)

['WLK2XZDqn98NRgU3nPcC', 'xkY8yRZc6lN1nBgJRqHe', 'lnUOu3hbHEogtaMBuDTe']

As a shorthand, you can initilize and add vectors in a single step using the `from_texts` and `from_documents` method.

In [12]:
vector_store = FirestoreVectorStore.from_texts(
    client=client,
    source="fruits",
    texts=fruits_texts,
    embedding=embedding,
)

In [14]:
vector_store = FirestoreVectorStore.from_documents(
    client=client,
    source="fruits",
    documents=fruits_docs,
    embedding=embedding,
)

## Similarity Search

You can use the `FirestoreVectorStore` to perform similarity searches on the vectors you have stored. This is useful for finding similar documents or text.

In [15]:
vector_store.similarity_search('{"name": "fuji"}', 3)

[Document(page_content='{"name": "apple"}', metadata={'reference': {'path': 'fruits/1m0hBFYZN6GOFcCncJ0t', 'firestore_type': 'document_reference'}, 'embeddings': Vector<0.03514678,-0.010442329,-0.0764273,-0.006872896,0.025266444,0.028062137,0.023098007,-0.01628814,-0.0046552815,0.02306933,-0.028545475,0.008150654,-0.015768338,0.018839134,0.003919609,-0.032474216,0.028655568,0.01971195,0.012483158,-0.05076017,-0.021570602,0.0040420503,-0.030796647,0.018920314,-0.0054875542,0.022895796,0.018512288,-0.044334117,-0.03131682,0.026685614,-0.014574454,0.030448614,-0.086053126,-0.0006311761,0.0027955067,-0.048132118,-0.012347424,0.011178467,0.02692418,0.0014646393,0.010604779,-0.05754825,-0.045419123,0.0049210982,0.018648542,-0.0005615061,-0.015920175,0.02205152,-0.015446501,-0.042948753,0.053735696,0.01135631,0.05875467,-0.04464473,-0.025630714,-0.020144299,0.0403371,-0.0010588698,-0.019344317,0.030873923,0.009754483,0.041301653,0.02432717,0.010436554,-0.014051022,-0.08182345,-0.014814747,0.0

In [16]:
vector_store.max_marginal_relevance_search('{"name": "gala"}', 5)

[Document(page_content='{"content": "{\\"name\\": \\"banana\\"}"}', metadata={'reference': {'path': 'fruits/IGy6jupKsSlYxcrXMGM9', 'firestore_type': 'document_reference'}}),
 Document(page_content='{"content": "{\\"name\\": \\"apple\\"}"}', metadata={'reference': {'path': 'fruits/1m0hBFYZN6GOFcCncJ0t', 'firestore_type': 'document_reference'}}),
 Document(page_content='{"content": "{\\"name\\": \\"orange\\"}"}', metadata={'reference': {'path': 'fruits/9TgAupcxE5O146sziS5w', 'firestore_type': 'document_reference'}}),
 Document(page_content='{"content": "{\\"name\\": \\"banana\\"}"}', metadata={'reference': {'path': 'fruits/UGUBtOeJ4qivC1Vc5LXw', 'firestore_type': 'document_reference'}}),
 Document(page_content='{"content": "{\\"name\\": \\"banana\\"}"}', metadata={'reference': {'path': 'fruits/WspdzbQ5e5Q7eYmbSd3G', 'firestore_type': 'document_reference'}})]